In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget


import pyspedas
from pyspedas import tplot, options, get_data
from data_class import LoadArtemis, plot_electron_spectrogram

import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from Training.data_loader import MagnetotailDataset
from Helper.silence import silence_all
from tqdm import tqdm
import numpy as np

# Create the CDAWeb interface object
cdaweb_obj = pyspedas.CDAWeb()

In [2]:
import pandas as pd

# Read the Excel file and extract the first sheet with last 4 columns
df = pd.read_excel('ART_Xing_times_pos.xlsx', sheet_name=0); satellite = 'THB'
df = df.iloc[:, -7:-1]  # Keep only the last 4 columns (last one is bugged so it is written with -5 to -1)


In [3]:
# Extracting first and last entries and first and last exits from the DataFrame
first_entries = df[df['first_entry'] == True]
last_entries  = df[df['last_entry']  == True]
first_exits   = df[df['first_exit']  == True]
last_exits    = df[df['last_exit']   == True]


In [4]:
mask = first_entries['coupled time'] > np.datetime64('2013-06-30T00:00:00')

In [5]:
# Convert the 'coupled time' column to a list for both last entries and first exits
first_entries_times = first_entries['coupled time'][first_entries['coupled time'] > np.datetime64('2013-06-30T00:00:00')].tolist()
last_entries_times = last_entries['coupled time'][last_entries['coupled time'] > np.datetime64('2013-06-30T00:00:00')].tolist()
first_exits_times = first_exits['coupled time'][first_exits['coupled time'] > np.datetime64('2013-06-30T00:00:00')].tolist()
last_exits_times = last_exits['coupled time'][last_exits['coupled time'] > np.datetime64('2013-06-30T00:00:00')].tolist()

In [6]:
first_entries_times

[Timestamp('2013-07-19 22:50:00')]

In [7]:
### Magnetotail time processing ###

# Increment each entry in last_entries_times by 30 minutes
last_entries_times = [time + pd.Timedelta(minutes=30) for time in last_entries_times]

# Decrease each entry in first_exits_times by 30 minutes
first_exits_times = [time - pd.Timedelta(minutes=30) for time in first_exits_times]

In [8]:
### Magnetosheath time processing ###

# Create a list of times 1 day before each entry in first_exits_times
first_entries_times_start = [time - pd.Timedelta(days=1) for time in first_entries_times]
first_entries_times_end = [time - pd.Timedelta(minutes=90) for time in first_entries_times]  # Use the original times for the end

# Convert the times to strings for both last entries and first exits
first_entries_times_start_str = [str(time) for time in first_entries_times_start]
first_entries_times_end_str = [str(time) for time in first_entries_times_end]

# Create aentries of times 1 day after each entry in last_entries_times
last_exits_times_start = [time + pd.Timedelta(minutes=90) for time in last_exits_times]  # Use the original times for the start
last_exits_times_end = [time + pd.Timedelta(days=1) for time in last_exits_times]

# Convert the times to strings for both last entries and first exits
last_exits_times_start_str = [str(time) for time in last_exits_times_start]
last_exits_times_end_str = [str(time) for time in last_exits_times_end]



In [9]:
# Convert the lists of times to strings
last_entries_times_str = [str(time) for time in last_entries_times]
first_exits_times_str = [str(time) for time in first_exits_times]

# Check if the lengths of the two lists are equal
if len(last_entries_times_str) == len(first_exits_times_str):
    print("The lists have the same length.")
else:
    print("The lists have different lengths. This will cause issues in the next step.")

The lists have the same length.


In [10]:
last_exits_times_end_str

['2013-07-25 21:45:00']

In [11]:
first_entries_times_start_str[0]


'2013-07-18 22:50:00'

In [12]:
# Add all magnetosheath data to the HDF5 file

# Add magnetosheath parts from before magnetotail entries
for i in tqdm(range(len(first_entries_times_start_str))):

    curr_data_class = LoadArtemis(start_time = first_entries_times_start_str[i], 
                                    end_time   = last_exits_times_end_str[i],
                                    satellite_list = [satellite] 
                                    )

    curr_data_class.convert_min_delta_all(start_time        = first_entries_times_start_str[i],
                                            end_time          = last_exits_times_end_str[i],
                                            dt_method         ='max', 
                                            custom_time_delta = 600)

    curr_data_class.extract_to_HDF5('testing_data/artemis_P1_v4.h5')


  0%|          | 0/1 [00:00<?, ?it/s]

Variable 'B_electric_field_gsm' contains only NaN values and has been removed.


100%|██████████| 1/1 [00:13<00:00, 13.47s/it]

Session session_0031 saved with data from 2013-07-18T22:50:00.000000000 to 2013-07-25T21:45:00.000000000


In [25]:

# Add magnetosheath parts from after magnetotail exits
for i in tqdm(range(len(last_exits_times_start_str))):

    with silence_all():
        curr_data_class = LoadArtemis(start_time = last_exits_times_start_str[i], 
                                    end_time   = last_exits_times_end_str[i],
                                      satellite_list = [satellite]
                                      )

        curr_data_class.convert_min_delta_all(start_time        = last_exits_times_start_str[i],
                                            end_time          = last_exits_times_end_str[i],
                                            dt_method         ='max', 
                                            custom_time_delta = 600)

        curr_data_class.extract_to_HDF5('testing_data/artemis_P1_v4.h5')

100%|██████████| 1/1 [00:06<00:00,  7.00s/it]


In [ ]:
# Add all magnetotail data to the HDF5 file
for i in tqdm(range(len(last_entries_times_str))):

    with silence_all():
        curr_data_class = LoadArtemis(start_time = last_entries_times_str[i], 
                                      end_time   = first_exits_times_str[i],
                                      satellite_list = [satellite]
                                      )

        # curr_data_class.plot_pyspedas('B_ion_eflux')

        curr_data_class.convert_min_delta_all(start_time          = last_entries_times_str[i],
                                                end_time          = first_exits_times_str[i],
                                                dt_method         ='max', 
                                                custom_time_delta = 600)

        curr_data_class.extract_to_HDF5('testing_data/artemis_P1_v4.h5')



100%|██████████| 23/23 [03:01<00:00,  7.88s/it]


In [ ]:
# TRAINING DATASET
file_path = Path('..') / "Data" / "training_data" / "artemis_P1_v4.h5"
abs_file_path = file_path.resolve()

# Create dataset
dataset = MagnetotailDataset(abs_file_path)



Setting trainable samples for each section...


100%|██████████| 137/137 [00:00<00:00, 371.92it/s]


Computing plasma beta for each section...


100%|██████████| 137/137 [00:02<00:00, 49.51it/s]


Extracting magnetic field x-component for each section...


100%|██████████| 137/137 [00:00<00:00, 68488.64it/s]


Computing magnetic field magnitude for each section...


100%|██████████| 137/137 [00:00<00:00, 60844.94it/s]
